# GPTQ(generalized post tranining quantization) vs GGML(Generative Graphical Model) vs GGUF(GPT generate unified format) vs AWQ(Activation aware weight quantization)

# PTQ(Post Training Quantization)
# QAT(Quantization Aware trainig)

GGUF is a new file format for the LLMs created with GGML library, which was announced in August 2023. GGUF is a highly efficient improvement over the GGML format that offers better tokenization, support for special tokens and better metadata storage.

GGUF and GGML are both file formats used to store machine learning models for inference. GGUF is a newer, more efficient version of GGML that was announced in August 2023. GGUF is designed to be faster to load and save models, and to be easier to read. It also includes more data about the model, such as prompt templates and support for special tokens, which makes it easier to support multiple architectures.

https://news.ycombinator.com/item?id=39482895#:~:text=GGML%20and%20GGUF%20are%20the,to%20an%208%20bit%20GGUF.

https://www.databricks.com/blog/ggml-gguf-file-format-vulnerabilities#:~:text=%22GGUF%20is%20a%20file%20format,GGUF%20for%20use%20in%20GGML.%22


https://medium.com/@charles.vissol/gguf-in-details-8a9953ac7883#:~:text=GGUF%20is%20a%20new%20file%20format%20for,special%20tokens%20and%20better%20metadata%20storage.%20Note.

# LLAMA CPP for QUANTIZATION

In [ ]:
! git clone https://github.com/ggerganov/llama.cpp

Cloning into 'llama.cpp'...
remote: Enumerating objects: 25402, done.
remote: Counting objects: 100% (6277/6277), done.
remote: Compressing objects: 100% (269/269), done.
remote: Total 25402 (delta 6126), reused 6032 (delta 6008), pack-reused 19125
Receiving objects: 100% (25402/25402), 45.18 MiB | 16.08 MiB/s, done.
Resolving deltas: 100% (18060/18060), done.


In [ ]:
!cd llama.cpp && LLAMA_CUBLAS=1 make && pip install -r requirements/requirements-convert-hf-to-gguf.txt
# install requriments.txt

## https://huggingface.co/docs/huggingface_hub/en/guides/download

In [ ]:
from huggingface_hub import snapshot_download

In [ ]:
model_name="Qwen/Qwen1.5-1.8B" # model from HF

#now we r going to create 2 dir, i.e one to keep this model and second dir to keep quantize model

In [ ]:
base_model="./original_model" #one to keep this model

In [ ]:
quantized_path = "./quantized_model" # second dir to keep quantize model

In [ ]:
snapshot_download(repo_id=model_name,local_dir=base_model,local_dir_use_symlinks=False) # to download the model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1194: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.79k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/7.28k [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

'/content/original_model'

In [ ]:
quantized_model_path=quantized_path+'/FP16.gguf'

In [ ]:
quantized_model_path

'./quantized_model/FP16.gguf'

In [ ]:
!mkdir quantized_model

In [ ]:
!python llama.cpp/convert-hf-to-gguf.py ./original_model/ --outtype f16 --outfile ./quantized_model/FP16.gguf

# in llama.cpp folder there will be convert-hf-to-gguf.py file , this file help u to convert your model into gguf
# ./original_model/ = second dir i.e for original model
# --outfile ./quantized_model/FP16.gguf = o/p file here we r going to store our converted model
# now ur going to get new dir of converted model and name of dir is quantized_model in that their will be FP16.gguf (converted file)

In [ ]:
# as of now our model is of 16bit quantization = 2 byte
# and now we want it in 4bit quantization

In [ ]:
methods=["q4_k_m"] # to convert it into 4bit = 0.5 byte

In [ ]:
quantized_path = "./quantized_model"

In [ ]:
import os

for method in methods:
  qtype=f"{quantized_path}/{method.upper()}.gguf"
  print(qtype)

./quantized_model/Q4_K_M.gguf


In [ ]:
import os

for method in methods:
  qtype=f"{quantized_path}/{method.upper()}.gguf"
  os.system("./llama.cpp/quantize "+ "./quantized_model"+"/FP16.gguf "+ qtype + " " + method)

# /llama.cpp  (dir) ==> go in quantize file
# ./quantized_model =(dir) ==> go in FP16.gguf file
# qtype variable define ago (in which type we have to convert i.e 16bit into 4 bit)


In [ ]:
! /content/llama.cpp/main -m ./quantized_model/Q4_K_M.gguf -n 90 --repeat_penalty 1.0 --color -i -r "User:" -f llama.cpp/prompts/chat-with-bob.txt


In [ ]:
"./llama.cpp/quantize "+ quantized_model+ "/FP16.gguf "+ qtype + " " + method

'./llama.cpp/quantize /quantized_model/FP16.gguf ./quantized_model/Q4_K_M.gguf q4_k_m'

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
# to upload ur model under ur huggingface hub

In [ ]:
from huggingface_hub import HfApi, HfFolder, create_repo, upload_file

In [ ]:
model_path="/content/quantized_model/Q4_K_M.gguf"

In [ ]:
repo_name="qwen1.5-llm-quantized"
# to push model on ur hugging face

In [ ]:
repo_url=create_repo(repo_name,private=False)

In [ ]:
api=HfApi()

In [ ]:
api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo="Q4_K_M.gguf", # model
    repo_id="snshrivas10/qwen1.5-llm-quantized", #repo in hugging face
    repo_type="model",

)

Q4_K_M.gguf:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/snshrivas10/qwen1.5-llm-quantized/commit/d2c4bbf58bce01fc08dfbc7c537ee981cc34273f', commit_message='Upload Q4_K_M.gguf with huggingface_hub', commit_description='', oid='d2c4bbf58bce01fc08dfbc7c537ee981cc34273f', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
3.67-1.22

In [1]:
#3.67-1.22
# 3.67gb is size of ur first model of 16bit
#1.22gb is size of converted model of 4bit
#we have converted our model and reduce file size near to 2.45gb
# we have performed quantization and can use this model locally anywhere.

## download the llama cpp in your local and use this quantized model over there

### llama model or mistral model and convert it into gguf or ggml format

### write a difference between gguf vs ggml vs gptq vs awq

### give me at least 5 differnce with proof

### PTQ(Post Training Quantization)
### QAT(Quantization Aware trainig)

sunny.savita@ineuron.ai

In [ ]:
#RLHF

In [ ]:
# QLORA And LORA

In [ ]:
# with project
1. aws
2. gcp
3. azure

In [ ]:
2-3 complete local setup

In [ ]:
# prompt engineering

In [ ]:
# invertiew questions

In [ ]:
LORA
QUANTIZATIOn

Q+L=QLORA